In [3]:
!pip install pandas
!pip install numpy
!pip install bs4
!pip install requests

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup



headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
           'Accept-Language': 'en-US, en;q=0.5'})


movie_name = []
year = []
time=[]
rating=[]
metascore =[]
director=[]
votes = []
gross = []
description = []
genre=[]
cast=[]
cas=[]
pages = np.arange(1,1000,50)
#https://www.imdb.com/search/title/?title_type=feature&primary_language=en
#https://www.imdb.com/search/title/?title_type=feature&primary_language=en&ref_=adv_prv
for page in pages:
   
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&primary_language=en&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace('(', '')
        year_of_release=year_of_release.replace(')','')
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.find('span', class_ = "runtime") else "NA"
        time.append(runtime)
        
        gen = store.p.find("span", class_ = 'genre').text
        genre.append(gen)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '') if store.find('div', class_ = "inline-block ratings-imdb-rating") else "NA"
        rating.append(rate)
        #rate = store.find('div', class_ = "ratings-bar").find('strong').text.replace('\n', '')
        #rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "NA"#if meta score not present then *
        
        metascore.append(meta)
        
        #dire=store.find('p',class_ = "metascore")
        dire=store.find('p',class_='').find_all('a')[0].text
        
        director.append(dire)
        
        #cas=([a.text for a in store.find('p',class_='').find_all('a')[1:]])
        #cast=','.join(map(str,cas))
        cast.append([a.text for a in store.find('p',class_='').find_all('a')[1:]])
      
        value = store.find_all('span', attrs = {'name':'nv'}) if store.find_all('span', attrs = {'name':'nv'}) else 'NA'
        vote = value[0].text if store.find_all('span', attrs = {'name':'nv'}) else 'NA'

        votes.append(vote)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else 'NA'
        description.append(description_)
        
#dataframe
for i in cast:
    c=','.join(map(str,i))
    cas.append(c)
   
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Genre":genre,"Movie Rating": rating, "Metascore of movie": metascore,"Director":director,"Cast":cas,"Votes" : votes,"Description": description})
movie_list.to_excel("movie data_new1.xlsx")

In [5]:
movie_list.head()

,Movie Name,Year of Release,Watch Time,Genre,Movie Rating,Metascore of movie,Director,Cast,Votes,Description
0,Eternals,2021,156 min,"\nAction, Adventure, Fantasy",6.8,52,Chloé Zhao,"Gemma Chan,Richard Madden,Angelina Jolie,Salma...","98,915","The saga of the Eternals, a race of immortal b..."
1,Dune: Part One,2021,155 min,"\nAction, Adventure, Drama",8.2,74,Denis Villeneuve,"Timothée Chalamet,Rebecca Ferguson,Zendaya,Osc...","336,049",Feature adaptation of Frank Herbert's science ...
2,The Harder They Fall,2021,139 min,"\nAction, Drama, Western",6.6,68,Jeymes Samuel,"Jonathan Majors,Zazie Beetz,RJ Cyler,Edi Gathegi","33,459",When an outlaw discovers his enemy is being re...
3,Red Notice,2021,118 min,"\nComedy, Crime, Thriller",6.4,39,Rawson Marshall Thurber,"Dwayne Johnson,Ryan Reynolds,Gal Gadot,Ritu Arya","99,320",An Interpol agent tracks the world's most want...
4,No Time to Die,2021,163 min,"\nAction, Adventure, Thriller",7.4,68,Cary Joji Fukunaga,"Daniel Craig,Ana de Armas,Rami Malek,Léa Seydoux","197,621",James Bond has left active service. His peace ...


In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle



data=pd.read_excel("movie data_new1.xlsx")
data.info()
data.rename(columns={'Unnamed: 0': 'movie_id'}, inplace=True)

columns=['Cast','Director','Genre','Movie Name','Description']


data[columns].isnull().values.any()#no null values

def get_important_features(data):
    important_features=[]
    for i in range (0,data.shape[0]):
        important_features.append(data['Movie Name'][i]+' '+data['Director'][i]+' '+data['Genre'][i]+' '+data['Description'][i])
    return important_features


#creating a column to hold the combined strings
data['important_features']=get_important_features(data)




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1000 non-null   int64  
 1   Movie Name          1000 non-null   object 
 2   Year of Release     998 non-null    object 
 3   Watch Time          956 non-null    object 
 4   Genre               1000 non-null   object 
 5   Movie Rating        943 non-null    float64
 6   Metascore of movie  912 non-null    float64
 7   Director            1000 non-null   object 
 8   Cast                1000 non-null   object 
 9   Votes               943 non-null    object 
 10  Description         1000 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 86.1+ KB


In [20]:
data

,movie_id,Movie Name,Year of Release,Watch Time,Genre,Movie Rating,Metascore of movie,Director,Cast,Votes,Description,important_features
0,0,Eternals,2021,156 min,"\nAction, Adventure, Fantasy",6.8,52.0,Chloé Zhao,"Gemma Chan,Richard Madden,Angelina Jolie,Salma...","98,915","The saga of the Eternals, a race of immortal b...","Eternals Chloé Zhao \nAction, Adventure, Fanta..."
1,1,Dune: Part One,2021,155 min,"\nAction, Adventure, Drama",8.2,74.0,Denis Villeneuve,"Timothée Chalamet,Rebecca Ferguson,Zendaya,Osc...","336,049",Feature adaptation of Frank Herbert's science ...,"Dune: Part One Denis Villeneuve \nAction, Adve..."
2,2,The Harder They Fall,2021,139 min,"\nAction, Drama, Western",6.6,68.0,Jeymes Samuel,"Jonathan Majors,Zazie Beetz,RJ Cyler,Edi Gathegi","33,459",When an outlaw discovers his enemy is being re...,"The Harder They Fall Jeymes Samuel \nAction, D..."
3,3,Red Notice,2021,118 min,"\nComedy, Crime, Thriller",6.4,39.0,Rawson Marshall Thurber,"Dwayne Johnson,Ryan Reynolds,Gal Gadot,Ritu Arya","99,320",An Interpol agent tracks the world's most want...,"Red Notice Rawson Marshall Thurber \nComedy, C..."
4,4,No Time to Die,2021,163 min,"\nAction, Adventure, Thriller",7.4,68.0,Cary Joji Fukunaga,"Daniel Craig,Ana de Armas,Rami Malek,Léa Seydoux","197,621",James Bond has left active service. His peace ...,"No Time to Die Cary Joji Fukunaga \nAction, Ad..."
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Pirates of the Caribbean: Dead Man's Chest,2006,151 min,"\nAction, Adventure, Fantasy",7.3,53.0,Gore Verbinski,"Johnny Depp,Orlando Bloom,Keira Knightley,Jack...","680,206",Jack Sparrow races to recover the heart of Dav...,Pirates of the Caribbean: Dead Man's Chest Gor...
996,996,Klaus,2019,96 min,"\nAnimation, Adventure, Comedy",8.2,65.0,Sergio Pablos,"Carlos Martínez López,Jason Schwartzman,J.K. S...","122,474",A simple act of kindness always sparks another...,"Klaus Sergio Pablos \nAnimation, Adventure, Co..."
997,997,Lone Survivor,2013,121 min,"\nAction, Biography, Drama",7.5,60.0,Peter Berg,"Mark Wahlberg,Taylor Kitsch,Emile Hirsch,Ben F...","277,206",Marcus Luttrell and his team set out on a miss...,"Lone Survivor Peter Berg \nAction, Biography, ..."
998,998,How to Be Single,2016,110 min,"\nComedy, Drama, Romance",6.1,51.0,Christian Ditter,"Dakota Johnson,Rebel Wilson,Leslie Mann,Alison...","84,871",A group of young adults navigate love and rela...,"How to Be Single Christian Ditter \nComedy, Dr..."


In [21]:
tfidf = TfidfVectorizer(stop_words='english')
#data['Description'] = data['Description'].fillna('')
tfidf_matrix = tfidf.fit_transform(data['important_features'])
tfidf_matrix.shape


cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)




In [22]:
cosine_sim


array([[1.        , 0.01227222, 0.00673544, ..., 0.00464334, 0.        ,
        0.07470759],
       [0.01227222, 1.        , 0.01065916, ..., 0.00734832, 0.00601658,
        0.01013311],
       [0.00673544, 0.01065916, 1.        , ..., 0.00849923, 0.00695891,
        0.01172019],
       ...,
       [0.00464334, 0.00734832, 0.00849923, ..., 1.        , 0.0047974 ,
        0.00807978],
       [0.        , 0.00601658, 0.00695891, ..., 0.0047974 , 1.        ,
        0.00661548],
       [0.07470759, 0.01013311, 0.01172019, ..., 0.00807978, 0.00661548,
        1.        ]])

In [23]:
indices = pd.Series(data.index, index=data['Movie Name']).drop_duplicates()
#indices['Stillwater']
#sim_scores = list(enumerate(cosine_sim[indices['Stillwater']]))
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar movies
    movies=data['Movie Name'].iloc[movie_indices]
    id=data['movie_id'].iloc[movie_indices]
    dict={"Movies":movies,"id":id}
    final_df=pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df


In [24]:
get_recommendations('Goodfellas')

,Movies,id
0,Married to the Mob,991
1,The Wolf of Wall Street,86
2,The Irishman,379
3,Notting Hill,957
4,Pulp Fiction,126


In [25]:
get_recommendations('Spider-Man: Far from Home')

,Movies,id
0,Spider-Man: No Way Home,8
1,Spider-Man: Homecoming,150
2,Spider-Man: Into the Spider-Verse,216
3,Spider-Man 4: Fan Film,460
4,The Amazing Spider-Man 2,426


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1000 non-null   int64  
 1   Movie Name          1000 non-null   object 
 2   Year of Release     998 non-null    object 
 3   Watch Time          956 non-null    object 
 4   Genre               1000 non-null   object 
 5   Movie Rating        943 non-null    float64
 6   Metascore of movie  912 non-null    float64
 7   Director            1000 non-null   object 
 8   Cast                1000 non-null   object 
 9   Votes               943 non-null    object 
 10  Description         1000 non-null   object 
 11  important_features  1000 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 93.9+ KB


In [28]:
new = data.drop(columns=['Year of Release','Watch Time','Genre','Movie Rating','Metascore of movie','Director','Cast','Votes','Description'])

In [29]:
new

,movie_id,Movie Name,important_features
0,0,Eternals,"Eternals Chloé Zhao \nAction, Adventure, Fanta..."
1,1,Dune: Part One,"Dune: Part One Denis Villeneuve \nAction, Adve..."
2,2,The Harder They Fall,"The Harder They Fall Jeymes Samuel \nAction, D..."
3,3,Red Notice,"Red Notice Rawson Marshall Thurber \nComedy, C..."
4,4,No Time to Die,"No Time to Die Cary Joji Fukunaga \nAction, Ad..."
...,...,...,...
995,995,Pirates of the Caribbean: Dead Man's Chest,Pirates of the Caribbean: Dead Man's Chest Gor...
996,996,Klaus,"Klaus Sergio Pablos \nAnimation, Adventure, Co..."
997,997,Lone Survivor,"Lone Survivor Peter Berg \nAction, Biography, ..."
998,998,How to Be Single,"How to Be Single Christian Ditter \nComedy, Dr..."


In [30]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(cosine_sim,open('similarity.pkl','wb'))

# Designing a front end

Now that we have completed building a model, we can start creating a web application.

Our model’s frontend uses Streamlit. Streamlit is an open-source Python library that makes it easy to build beautiful custom web apps for machine learning and data science.

## pip install streamlit

There are already many good tutorials on streamlit like this one. So I don’t want to go into detail on all my processes. In general, my goal was to build a frontend that gets user input and recommends a list of similar movies.